In [24]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

import cooler
import bioframe
import cooltools
from cooltools.lib.numutils import fill_diag
from pybedtools import BedTool as pbt

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
from itertools import chain
import warnings
import logging

warnings.simplefilter(action='ignore', category=FutureWarning)

In [25]:
from dotenv import load_dotenv
assert os.environ['CONDA_DEFAULT_ENV'] == "cultures_hic"
load_dotenv()

True

In [15]:
chromnames = ['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22',
 'chrX']

# 1. Load maps

In [11]:
def load_maps(path_to_maps, pattern = '.mcool', addtitional_pattern = None):
    files = [f for f in listdir(path_to_maps) if pattern in f]
    if addtitional_pattern:
        files = [f for f in listdir(path_to_maps) if addtitional_pattern in f]
    return files

def get_chromosomes():
    hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
    hg38_cens = bioframe.fetch_centromeres('hg38')
    hg38_arms = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
    return hg38_arms.set_index("chrom").loc[chromnames].reset_index()

def get_expected_map(clr, name_exp, binsize, save_path = None):
    exp = cooltools.expected_cis(
    clr,
    view_df=hg38_arms,
    nproc=18)
    if save_path:
        exp.to_pickle(f"{save_path}/{name_exp}_{binsize}res.pickle")
    return exp

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_or_compute_expected_map(map_name, path_to_maps, path_to_maps_expected, hg38_arms, binsize=15_000, nproc=14):
    """
    Load expected map from a pickle file or compute it if not available.

    Parameters:
    - map_name: str, name of the map file.
    - path_to_maps: str, path to the directory containing map files.
    - path_to_maps_expected: str, path to the directory for storing expected map pickles.
    - hg38_arms: DataFrame, view dataframe for cooler.
    - binsize: int, bin size for cooler.
    - nproc: int, number of processes to use.

    Returns:
    - expected_per_chrArm: DataFrame, expected values per chromosome arm.
    """
    map_prefix = map_name.split('.mcool')[0]
    pickle_file = os.path.join(path_to_maps_expected, f'{map_prefix}_perChrArm.pickle')
    
    if os.path.exists(pickle_file):
        logging.info(f"Loading expected map from {pickle_file}")
        expected_per_chrArm = pd.read_pickle(pickle_file)
    else:
        logging.info(f"Computing expected map for {map_name}")
        clr = cooler.Cooler(f'{path_to_maps}/{map_name}::/resolutions/{binsize}')
        expected_per_chrArm = cooltools.expected_cis(clr, view_df=hg38_arms, nproc=nproc)
        expected_per_chrArm.to_pickle(pickle_file)
        logging.info(f"Expected map saved to {pickle_file}")
    
    return expected_per_chrArm

def process_maps(files, path_to_maps, path_to_maps_expected, hg38_arms, binsize=15_000, nproc=14):
    """
    Process a list of map files to load or compute expected maps.

    Parameters:
    - files: list of str, names of the map files.
    - path_to_maps: str, path to the directory containing map files.
    - path_to_maps_expected: str, path to the directory for storing expected map pickles.
    - hg38_arms: DataFrame, view dataframe for cooler.
    - binsize: int, bin size for cooler.
    - nproc: int, number of processes to use.

    Returns:
    - expected_maps: dict, map of filenames to their expected map DataFrames.
    """
    expected_maps = {}
    for map_name in files:
        logging.info(f"Processing map: {map_name}")
        try:
            expected_maps[map_name] = load_or_compute_expected_map(
                map_name, path_to_maps, path_to_maps_expected, hg38_arms, binsize, nproc
            )
        except Exception as e:
            logging.error(f"Failed to process {map_name}: {e}")
    
    return expected_maps

In [28]:
path_to_maps = os.getenv('PATH_TO_MAPS')
path_to_maps_expected = os.getenv('PATH_TO_EXPECTED_MAPS')
path_to_custom_kernels = os.getenv('PATH_TO_CUSTOM_KERNELS')

number_of_files = 23
hg38_arms = get_chromosomes()

In [23]:
files = load_maps(path_to_maps)
assert len(files) == number_of_files
expected_maps = process_maps(files, path_to_maps, path_to_maps_expected, hg38_arms)

Ballerino2022_NES_5kb.drop_diag.5kb.mcool.sampled_exact.mcool
Lu2020_iPSC_NeuNplus.sampled_exact.mcool
Rahman2023_FetalBrain.drop_diag.5kb.mcool.sampled_exact.mcool
Zaghi2023_iPSC_NeuNplus.sampled_exact.mcool
Hu2021_NeuNplus.sampled_exact.mcool
Ballerino2022_iPSC_NeuNplus.sampled_exact.mcool
Rahman2023_NeuNplus.sampled_exact.mcool
Lu2020_iPSC_5kb.drop_diag.5kb.mcool.sampled_exact.mcool
Our_data_iPSC_NeuNplus.drop_diag.5kb.mcool.sampled_exact.mcool
Heffel_infant.3056_cells.5kb.drop_diag.5kb.sampled_exact.mcool
Wu2021_iPSC_NeuNplus.sampled_exact.mcool
Tian2023_NeuNplus.EN_IN.29_42_58_years.2000_cells.sampled_exact.mcool
Ballerino2022_NPC_5kb.drop_diag.5kb.mcool.sampled_exact.mcool
Heffel_3T.3056_cells.5kb.drop_diag.5kb.sampled_exact.mcool
Heffel_2T.3056_cells.5kb.drop_diag.5kb.sampled_exact.mcool
Rajarajan_iPSC_NeuNplus.sampled_exact.mcool
Pletenev2024_NeuNplus.sampled_exact.mcool
Rahman2023_iPSC_NeuNplus_CRISPRi_Scrambled_A_DpnII-HinfI.sampled_exact.mcool
Rajarajan_iPSC_Glia_5kb.drop_di

# 2. Define custom kernels

In [30]:
def load_kernels():
    kernels_narrow = np.load(f'{path_to_custom_kernels}/kernels_narrow.npy' ,allow_pickle=True)
    kernels_wide = np.load(f'{path_to_custom_kernels}/kernels_wide.npy' ,allow_pickle=True)
    kernels_super_wide = np.load(f'{path_to_custom_kernels}/kernels_super_wide.npy' ,allow_pickle=True)
    
    kernels_narrow = dict(enumerate(kernels_narrow.flatten()))[0]
    kernels_wide = dict(enumerate(kernels_wide.flatten()))[0]
    kernels_super_wide = dict(enumerate(kernels_super_wide.flatten()))[0]
    
    return kernels_narrow, kernels_wide, kernels_super_wide

In [31]:
kernels_narrow, kernels_wide, kernels_super_wide = load_kernels()


# 3. Call chromatin loops

In [40]:
max_loci_separation_set = 12000000
def get_dots_standard(clr, exp, fdr, nans):
    dots = cooltools.dots(
        clr,
        expected=exp,
        view_df=hg38_arms,   
        max_loci_separation = max_loci_separation_set,
        max_nans_tolerated=nans,  
        lambda_bin_fdr=fdr,
        clustering_radius=binsize*2.5,
        tile_size=7_000_000,   
        nproc=14,
    )
    
    return dots

def get_dots_customkernel(clr, exp,  fdr, nans):
    dots_customkernel = cooltools.dots(
    clr,
    expected=exp,
    view_df=hg38_arms,   
    max_loci_separation=max_loci_separation_set,
    max_nans_tolerated=nans,  
    kernels = kernels_wide,
    clustering_radius=binsize*2.5,
    lambda_bin_fdr=fdr,
    tile_size=7_000_000,
    nproc=14,
)
    return dots_customkernel


def get_dots_customkernelCircle(clr, exp, fdr, nans):
    dots_customkernelCircle = cooltools.dots(
    clr,
    expected=exp,
    view_df=hg38_arms,   
    max_loci_separation=max_loci_separation_set,
    max_nans_tolerated=nans,  
    kernels = kernels_super_wide,
    clustering_radius=binsize*2.5,
    lambda_bin_fdr=fdr,
    tile_size=7_000_000,
    nproc=14,
)
    return dots_customkernelCircle


def get_dots_customkernelSmall(clr, exp, fdr, nans):
    dots_customkernelCircle = cooltools.dots(
    clr,
    expected=exp,
    view_df=hg38_arms,   
    max_loci_separation=max_loci_separation_set,
    max_nans_tolerated=nans,  
    kernels = kernels_narrow,
    clustering_radius=binsize*2.5,
    lambda_bin_fdr=fdr,
    tile_size=7_000_000,
    nproc=14,
)
    return dots_customkernelCircle

In [41]:
def prep_df(dots_clr_HCplus_10_merged_customkernelExtreme):
    df = dots_clr_HCplus_10_merged_customkernelExtreme.iloc[:,:6]
    df["num"] = [i for i in range(df.shape[0])]
    return df

def process_pair_to_pair(source_df, target_df, slope):
    result = pbt.from_dataframe(source_df).pair_to_pair(pbt.from_dataframe(target_df), slop=slope)
    if os.path.getsize(result.fn) > 0:
        non_unique_df = pd.read_table(result.fn, header=None)
        unique_df = source_df[~source_df["num"].isin(non_unique_df[6])]
        assert unique_df["num"].nunique() == source_df["num"].nunique() - non_unique_df[6].nunique(), "Uniqueness criteria failed"
        return unique_df
    else:
        return source_df.copy()

def make1_unique(df_customkernel, df_init, slope_factor):
    unique2_to_1 = process_pair_to_pair(df_customkernel, df_init, slope_factor)
    return unique2_to_1
        
def make2_unique(df_customkernelExtreme, df_init, df_customkernel, slope_factor):
    unique3_to_2 = process_pair_to_pair(df_customkernelExtreme, df_customkernel, slope_factor)
    unique3_to_1 = process_pair_to_pair(unique3_to_2, df_init, slope_factor)
    return unique3_to_1
    
def make3_unique(df_customkernelCircle, df_customkernelExtreme, df_customkernel, df_init, slope_factor):
    unique4_to_1 = process_pair_to_pair(df_customkernelCircle, df_init, slope_factor)
    unique4_to_2 = process_pair_to_pair(unique4_to_1, df_customkernel, slope_factor)
    unique4_to_1_2_3 = process_pair_to_pair(unique4_to_2, df_customkernelExtreme, slope_factor)
    return unique4_to_1_2_3
 

def create_unique_border(dots_clr_HCplus_10_merged, dots_clr_HCplus_10_merged_customkernel, dots_clr_HCplus_10_merged_customkernelExtreme, dots_customkernelCircle, binsize):
    slope_factor = binsize * 2.5
    
    df_customkernelExtreme = prep_df(dots_clr_HCplus_10_merged_customkernelExtreme)
    df_customkernel = prep_df(dots_clr_HCplus_10_merged_customkernel)
    df_init = prep_df(dots_clr_HCplus_10_merged)
    df_customkernelCircle = prep_df(dots_customkernelCircle)

    unique2_to_1 = make1_unique(df_customkernel, df_init, slope_factor)
    unique3_to_1 = make2_unique(df_customkernelExtreme, df_init, df_customkernel, slope_factor)
    unique4_to_1_2_3 = make3_unique(df_customkernelCircle, df_customkernelExtreme, df_customkernel, df_init, slope_factor)

    df_init['kernel'] = 'standard'
    unique2_to_1['kernel'] = 'wide'
    unique3_to_1['kernel'] = 'super_wide'
    unique4_to_1_2_3['kernel'] = 'narrow'

    union2 = pd.concat([df_init, unique3_to_1, unique2_to_1, unique4_to_1_2_3]).reset_index(drop=True)
    union2["num"] = list(range(union2.shape[0]))

    return union2


In [44]:
def get_all_loops(file, prep_maps, binsize=10000, fdr=0.16, nans = 5, temp_dir="./loops_data/loops_temp_files/", final_dir="./loops_data/loops_final_files"):
    
    exp = prep_maps[file]
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(final_dir, exist_ok=True)

    clr = cooler.Cooler(f'{path_to_maps}/{file}::/resolutions/{binsize}')
    name_exp = ("_").join(file.split('.')[:-1])
    print(file, name_exp)

    dots = get_dots_standard(clr, exp, fdr, nans)
    print(dots.shape[0])
    dots.to_feather(os.path.join(temp_dir, f"{name_exp}_dots_regular_{max_loci_separation_set}maxloci_{fdr}fdr_{binsize}res_small_NaN{nans}.feather"))

    # Custom kernel dots
    dot_custom = get_dots_customkernel(clr, exp, fdr, nans)
    print(dot_custom.shape[0])
    dot_custom.to_feather(os.path.join(temp_dir, f"{name_exp}_dots_custom_{max_loci_separation_set}maxloci_{fdr}fdr_{binsize}res_small_NaN{nans}.feather"))

    # Custom kernel circle dots
    dots_customkernelCircle = get_dots_customkernelCircle(clr, exp, fdr, nans)
    print(dots_customkernelCircle.shape[0])
    dots_customkernelCircle.to_feather(os.path.join(temp_dir, f"{name_exp}_dots_customkernelCircle_{max_loci_separation_set}maxloci_{fdr}fdr_{binsize}res_small_NaN{nans}.feather"))

    # Small custom kernel dots
    dot_custom_small = get_dots_customkernelSmall(clr, exp, fdr, nans)
    print(dot_custom_small.shape[0])
    dot_custom_small.to_feather(os.path.join(temp_dir, f"{name_exp}_dots_small_{max_loci_separation_set}maxloci_{fdr}fdr_{binsize}res_small_NaN{nans}.feather"))

    # Create unique border dots
    dots_final = create_unique_border(dots, dot_custom, dots_customkernelCircle, dot_custom_small, binsize)
    dots_final.to_feather(os.path.join(temp_dir, f"{name_exp}_dots_final_{max_loci_separation_set}maxloci_{fdr}fdr_{binsize}res_small_NaN{nans}.feather"))
    dots_final.to_csv(os.path.join(final_dir, f"{name_exp}_dots_final_{max_loci_separation_set}maxloci_{fdr}fdr_{binsize}res_small_NaN{nans}_final.bed"), sep="\t", header=None, index=False)
    
    return dots_final


In [46]:
binsize = 15_000
fdr = 0.13
binsize=15000

assert set(expected_maps.keys()) == set(files)
for file in files:
    final_loops = get_all_loops(file, expected_maps, binsize = binsize, fdr = fdr)
    name_exp = ("_").join(file.split('.')[:-1])
